In [4]:
# Kreieren von Leeren Listen um die Resultate zu sammeln
results = []
double_entries = []

# Definition der Betrachtungsdauer für die Monte Carlo
# 100 Jahre * 365 Tage * 24 Stunden = 876'000 Stunden

amount_of_hours = 876000

# Kreiere eine Liste um die die Resultate darin abzufüllen
for i in range (0, amount_of_hours, 1):
    results.append(i)

for i in range (0, amount_of_hours, 1):
    double_entries.append(i)
    
# Transformation in ein Pandas DataFrame Object    
results = pd.DataFrame(results, columns= ['Hours'])
double_entries = pd.DataFrame(double_entries, columns= ['Hours'])

# Definition der benötigten Spalten
header = ['Day', 'Hour of Day', 'M 1', 'Speed 1', 'M 2', 'Speed 2', 'E in kJ 1', 'E in kJ 2', 'Sum M', 'Breakthrough']

# Hinzufügen aller Spalten ins DataFrame Object und befülle mit NaN Werten
for value in header:
    results[value] = np.nan
    
for value in header: 
    double_entries[value] = np.nan

In [5]:
# Start- und Endpunkt für die Monte Carlo Simulation
end = amount_of_hours
start = 0

# Iterationsschritt für Masse und Geschwindigkeit
i = 0
j = 0

# Fortlaufender Zähler der Zeit des Abschlags
event_1 = 0
event_2 = 0

In [6]:
# Berechnungen der Zufallsvariablen in der Monte Carlo Simulation
# Im ersten Schritt wird das Zeitdelta berechnet, ist das Zeitdelta > 0 dann wird die Masse und die Geschwindigkeit 
# als Zufallsvariable berechnet. Dabei wird die Funktion von weiter oben abgerufen. Im letzten Schritt werden Masse 
# und Geschwindigkeit zum Zeitpunkt "event_X" im DataFrame Objekt gespeichert. 

# Kommt es dazu, dass ein event zur gleichen Zeit zwei Mal vorkomme (2 Abschläge im gleichen Zeitenster, dann wird m 
# DataFrame "double_entries" abgespeichert (Bedingung t_delta = 0)

while i < end:
    t_delta_1 = random_t_delta_1()
    
    if t_delta_1 > 0:        
        event_1 += t_delta_1
        
        masse_1 = random_masse_1()
        results['M 1'][event_1] = masse_1
        
        speed_1 = random_speed_1()
        results['Speed 1'][event_1] = speed_1
        i += t_delta_1
        
    else: 
        masse_1 = random_masse_1()
        double_entries['M 1'][event_1] = masse_1
        
        speed_1 = random_speed_1()
        double_entries['Speed 1'][event_1] = speed_1
        
while j < end:
    t_delta_2 = random_t_delta_2()
    
    if t_delta_2 > 0:
        event_2 += t_delta_2
    
        masse_2 = random_masse_2()
        results['M 2'][event_2] = masse_2
    
        speed_2 = random_speed_2()
        results['Speed 2'][event_2] = speed_2
        j += t_delta_2
    
    else:
        masse_2 = random_masse_2()
        double_entries['M 2'][event_2] = masse_2
    
        speed_2 = random_speed_2()
        double_entries['Speed 2'][event_2] = speed_2 

c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [7]:
# Da es wenige Ereignisse gibt die im gleichen Zeitfesnter geschehen, können die überflüssigen Zeilen rausgefiltert 
# werden

filtered_double_entries = double_entries.dropna(thresh = 2)

In [8]:
# Tabelle zusammenfügen, bereinigen und auffüllen

frames = [results, filtered_double_entries]
all_data = pd.concat(frames)

all_data = all_data.sort_values(by = ['Hours'] )

all_data = all_data.fillna(0)

all_data = all_data.reset_index(drop = True)

In [9]:
# Berechnung fehlender Felder

# Formel für kinetische Energie verwenden
all_data['E in kJ 1'] = all_data.apply(lambda x: all_data['M 1'] * all_data['Speed 1'] * all_data['Speed 1'] / 2 / 1000)
all_data['E in kJ 2'] = all_data.apply(lambda x: all_data['M 2'] * all_data['Speed 2'] * all_data['Speed 2'] / 2 / 1000)

# Berechnung des Tages um eine bessere Kotrolle im Enddokument vornehmen zu können
all_data['Day'] = all_data.apply(lambda x: all_data['Hours'] // 24)

# Berechnung der Stunde des Tages um eine bessere Kontrolle im Enddokument vornehmen zu können
all_data['Hour of Day'] = all_data.apply(lambda x: all_data['Hours'] % 24)

In [10]:
# In diesem Schritt werden die Steine alle 24 Stunden geleert. Um dies zu bewerkstelligen, werden zur Stunde 0 die 
# Summen in den Netzen geleert und ansonsten immer mit dem vorherigen Element gefüllt.

INDEX = 0
end = all_data['Hours'].iloc[-1]
sum_M = 0

while INDEX <= end:

    modulo = all_data['Hours'][INDEX] % 24

    if modulo != 0:
        sum_M += all_data['M 1'][INDEX] + all_data['M 2'][INDEX]

        all_data['Sum M'][INDEX] = sum_M

        INDEX += 1

    else:
        sum_M = all_data['M 1'][INDEX] + all_data['M 2'][INDEX]
        all_data['Sum M'][INDEX] = sum_M
        INDEX += 1

c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [11]:
# Im nächsten Schritt wird beurteilt ob der runtergefallne Stein einen Durchbruch im Netz hervorruft. Ist dies der 
# Fall, dann wird in der Zelle "Breakthrough" eine 1 ( = True) geschrieben, ansonsten eine 0 ( = False)

INDEX = 0
end = all_data['Hours'].iloc[-1]

while INDEX < end:
    
    if all_data['Hour of Day'][INDEX] == 0:
        
        if (all_data['E in kJ 1'][INDEX] or all_data['E in kJ 2'][INDEX]) >= 1000:
            all_data['Breakthrough'][INDEX] = 1
            
            while True:
                INDEX += 1

                if all_data['Hour of Day'][INDEX] == 0:
                    break
        
        else:
            INDEX += 1
       
    else:
        if all_data['Sum M'][INDEX-1] < 2000:

            if (all_data['E in kJ 1'][INDEX] or all_data['E in kJ 2'][INDEX]) >= 1000:
                all_data['Breakthrough'][INDEX] = 1
                
                while True:
                    INDEX += 1
                    
                    if all_data['Hour of Day'][INDEX] == 0:
                        break
                        
            else:
                INDEX += 1


        elif all_data['Sum M'][INDEX-1] > 2000:

            if (all_data['E in kJ 1'][INDEX] or all_data['E in kJ 2'][INDEX]) >= 500:
                all_data['Breakthrough'][INDEX] = 1
                
                while True:
                    INDEX += 1
                    
                    if all_data['Hour of Day'][INDEX] == 0:
                        break

            else:
                INDEX += 1

writer = pd.ExcelWriter('output_per_hour.xlsx') 
all_data.to_excel(writer)
writer.save()

c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sy

In [12]:
# Kreiere ein Subset der Breakthroughs und speichere das Dokument um es später wieder verwenden zu können

subset = all_data[(all_data['Breakthrough'] > 0)]

In [13]:
writer = pd.ExcelWriter('breakthrough.xlsx') 
subset.to_excel(writer)
writer.save()